# Goal
1. for an incoming set or df or users needing their datarefreshed:
1. check the quality of those revisions via ores or flagging
1. figure out the dataformat for the revisons dable
1. sync revisons back to (another process)

In [1]:
def from_wmftimestamp(bytestring):
    if bytestring:
        s = bytestring.decode('utf-8')
        return dt.strptime(s, '%Y%m%d%H%M%S')
    else:
        return bytestring
    
def to_wmftimestamp(date):
    return date.strftime('%Y%m%d%H%M%S')
    
    

def decode_or_nan(b):
    return b.decode('utf-8') if b else float('nan')


import json
import requests

from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError
from pymysql.err import InternalError, OperationalError
import sys
import os
import pandas as pd
import numpy as np

import mwclient
import mwviews
import mwapi
import ores_api


from datetime import datetime as dt
from datetime import timedelta as td

%pylab inline

langs_non_de = ['ar', 'fa', 'pl']
langs = langs_non_de + ['de']

# site = mwclient.Site(('https', f'{langcode}.wikipedia.org'), path = '/w/')

# os.makedirs(datadir, exist_ok=True)

constr = 'mysql+pymysql://{user}:{pwd}@{host}:{port}'.format(user=os.environ['MYSQL_USERNAME'],
                                                      pwd=os.environ['MYSQL_PASSWORD'],
                                                      host=os.environ['MYSQL_HOST'],
                                                    port=os.environ['MYSQL_PORT'],
                                                         charset='utf8',
                                                        use_unicode=True)

con = create_engine(constr, encoding='utf-8')


con.execute(f'use enwiki_p;')


CACHE_ROOT = os.getenv('CACHE_DIR', './cache')
GRAT_ROOT = os.getenv('GRAT_DIR', '../gratitude/outputs/')

Populating the interactive namespace from numpy and matplotlib


KeyError: 'MYSQL_USERNAME'

In [2]:
import mwreverts.api
import mwapi

# We'll use the mwreverts API check.  In order to do that, we need an API session
de_session = mwapi.Session("https://de.wikipedia.org", 
                        user_agent="Max Klein Newcomer Quality <max@notconfusing.com>")

# for a single user, get their revisions and determine how many are flagged or not

def get_revisions_and_flagged_data(rev_ids, treatment_date=datetime.datetime.utcnow()):
    """get number of revisons flagged before `treatment_date`
      among last 50 edits"""
    rev_flag_sql = """
    select rev_id, 
            rev_page, 
            page_namespace, 
            rev_timestamp, 
            fr_timestamp, 
            (select max(fr_timestamp) from flaggedrevs where fr_page_id=rev_page and fr_timestamp < {treatment_date}) max_fr_ts 
        from (
              select rev_id, rev_page, rev_timestamp, page_namespace from revision_userindex
                join page on page_id = rev_page where rev_id in ({rev_ids_str}) ) auser
        left join flaggedrevs on
            fr_page_id = rev_page and
            fr_rev_id = rev_id;
                """.format(rev_ids_str=','.join([str(x) for x in rev_ids]), treatment_date=to_wmftimestamp(treatment_date))
    
    con.execute('use dewiki_p;')
    rev_flag = pd.read_sql(rev_flag_sql, con)
    rev_flag['fr_timestamp'] = rev_flag['fr_timestamp'].apply(from_wmftimestamp)
    rev_flag['max_fr_ts'] = rev_flag['max_fr_ts'].apply(from_wmftimestamp)
    rev_flag['rev_timestamp'] = rev_flag['rev_timestamp'].apply(from_wmftimestamp)
    return rev_flag

def was_reverted(rev_id):
    try:
        _, reverted, reverted_to = mwreverts.api.check(
            de_session, rev_id, radius=3,  # most reverts within 5 edits
            window=48*60*60,  # 2 days
            rvprop={'user', 'ids'})  # Some properties we'll make use of
        return True if reverted else False
    except (KeyError, mwapi.session.APIError) as err:
        print('Error getting revert status for {rev_id}'.format(rev_id=rev_id))
        return True #because even if it was deleted from the DB for our purposes its still a bad edit 

def decide_flagged(row):
    """Was this revision flagged (or generally high quality)?"""
    namespace = row['page_namespace']
    rev_time = row['rev_timestamp']
    flagged_time = row['fr_timestamp']
    last_flagged_time = row['max_fr_ts']
    was_reverted = row['was_reverted']
    
    #namespace check
    if namespace != 0:
        return True # because often user-page edits are never approved   
    # check if explictly flagged
    elif pd.notnull(flagged_time):
        return True
    #check if reverted
    elif was_reverted:
        return False
    #check if the last flagged time is after the edit
    elif last_flagged_time:
        if rev_time < last_flagged_time:
            return True
        #the revision exists but it hasn't been flagged yet.
        else:
            return False
    else:
        #not sure what else would get to this stage , but...
        return False
        
def get_flagged_decision_df(rev_ids, treatment_date):
    rev_df = get_revisions_and_flagged_data(rev_ids, treatment_date)
    rev_df['was_reverted'] = rev_df.apply(lambda row: was_reverted(row['rev_id']) if pd.isnull(row['fr_timestamp']) else 'no_check', axis=1)
    rev_df['flagged'] = rev_df.apply(decide_flagged, axis=1)
    return rev_df


def get_flagged_revs(rev_ids, treatment_date):
    needed_columns = ['user_id', 'rev_id', 'rev_timestamp','was_flagged', 'was_reverted']
    rev_df = get_flagged_decision_df(rev_ids, treatment_date)
    rev_df['quality_enough'] = rev_df.apply(decide_flagged, axis=1)
    return rev_df

# Get Revisions of Editors

In [111]:
def get_recent_edits(lang, user_id, prior_days=84, max_revs=50):
    '''this will get all the rev_ids for a user that occured less than `prior_days` days before their last edit
    and no more than `max_revs` edits in total'''
    con.execute('use {lang}wiki_p;'.format(lang=lang))
    revsql = '''
            select user_id, ts as rev_timestamp, rev_id from
            (select a.rev_user as user_id, timestamp(a.rev_timestamp) as ts, a.rev_id as rev_id, timestamp(b.mts) as mts
            from
            (select rev_user, rev_timestamp, rev_id from revision_userindex where rev_user = {user_id}) a
            join
            (select rev_user, max(rev_timestamp) as mts from revision_userindex where rev_user = {user_id})  b
            on a.rev_user = b.rev_user
            ) uhist
            where ts > date_sub(mts, interval {prior_days} day)
            limit {max_revs};'''.format(user_id=user_id, prior_days=prior_days, max_revs=max_revs)
    udf = pd.read_sql(revsql, con)
    return udf

In [163]:
def get_all_users_revs(refresh_users, lang):
    """
    of the users needing refresh, get their their recent edits
    refresh_users is a DF having columns lang and user_id
    """
    all_users_revs_dfs = []
    for user_id in refresh_users['user_id'].values :
        recent_users_revs = get_recent_edits(lang, user_id)
        recent_users_revs['lang'] = lang
        all_users_revs_dfs.append(recent_users_revs)
        
    all_users_revs = pd.concat(all_users_revs_dfs)
    return all_users_revs
        

In [164]:
def get_ores_data_dgf_from_api(rev_ids, context_lang):
    session = ores_api.Session(
        'https://ores.wikimedia.org',
        user_agent='CivilServant Experiment Sampler <max.klein@civilservant.io>',
        batch_size=50,
        parallel_requests=4,
        retries=2)
    
    context = f'{context_lang}wiki'
    return session.score(context, ('damaging', 'goodfaith'), rev_ids)
    return scores
    return rev_ids_scores


def ores_quality_getter(rev_ids, context_lang):
    scores = get_ores_data_dgf_from_api(rev_ids, context_lang)
#     print([s for s in scores])
    try:
        predictions = [(sc['damaging']['score']['prediction'], sc['goodfaith']['score']['prediction']) for sc in scores]
#         print(predictions)
        predictions_ndgf = [(not(d) and gf) for (d, gf) in predictions]
#         print(predictions_ndgf)
    except KeyError: #probably couldn't get the score for because it doesn't exist
        predictions_ndgf = [False for i in range(len(rev_ids))]
#     rev_ids_scores = dict(zip(rev_ids, predictions_ndgf))
 
    rev_ids_scores = pd.DataFrame.from_dict({'rev_id':rev_ids,'quality_enough':predictions_ndgf},orient='columns')
    rev_ids_scores['lang'] = context_lang
    return rev_ids_scores


def flagged_rev_quality_getter(rev_ids, context_lang):
    rev_ids_flagged = get_flagged_revs(rev_ids, treatment_date=datetime.datetime.utcnow())
    rev_ids_flagged['lang'] = context_lang
    return rev_ids_flagged


def remove_non_quality_revs(all_user_revs):
    """
    remove the revisions that aren't ORES good faith for ar/pl/fa
    or aren't flagged for de.
    """
    langs = list(set(all_user_revs['lang']))
    rev_ids = all_user_revs['rev_id'].values

    if len(langs) > 1:
        raise NotImplementedError('Please just process one language at a time for sanity sake')
    elif langs[0] in ['ar','pl','fa']:
        revs_quality = ores_quality_getter(rev_ids, langs[0])
    elif langs[0] in ['de']:
        revs_quality = flagged_rev_quality_getter(rev_ids, 'de')
    
    all_user_revs_quality = pd.merge(all_user_revs, revs_quality, how="left", on=['rev_id','lang'])
    quality_user_revs = all_user_revs_quality[all_user_revs_quality['quality_enough']==True]
    print(f'started with {len(all_user_revs)} revs, removed {len(all_user_revs)-len(quality_user_revs)}')
    return quality_user_revs

In [171]:
de_user = get_all_users_revs(pd.DataFrame({'user_id':[2760580]}), 'de')

In [273]:
def get_diff_html_dict(rev_id, mwapi_session):
    ret = mwapi_session.get(action='compare', fromrev=rev_id, torelative='prev', prop='diff|user|comment|rel|title|ids')
    return ret['compare']

def get_rev_dict(old_rev_id, new_rev_id, mwapi_session):
    #check case where old_id_doesnt exist because its the newest edit on page
    revids_str = f'{old_rev_id}|{new_rev_id}' if old_rev_id else new_rev_id 
    ret = mwapi_session.get(revids=revids_str, action='query', prop='revisions', rvprop='timestamp')
    page_key = list(ret['query']['pages'].keys())[0]
    rev_data = ret['query']['pages'][page_key]['revisions']
    if old_rev_id:
        return rev_data
    else:
        ({'timestamp': None}, rev_data[0])
    

def get_display_data(rev_ids, lang):
    display_data = []
    mwapi_session = mwapi.Session(host=f'https://{lang}.wikipedia.org', user_agent='civilservant datagathering <max@notconfusing.com>')
    for rev_id in rev_ids:
        try:
            diff = get_diff_html_dict(rev_id=rev_id, mwapi_session=mwapi_session)
            try:
                old_rev_id = diff['fromrevid']
            except KeyError: #this is the very first edit on the page
                old_rev_id = None
            old_rev_data, new_rev_data = get_rev_dict(old_rev_id=old_rev_id, new_rev_id=rev_id, mwapi_session=mwapi_session)
            display_datum = {'editDeleted':False, 'diffHTML':diff['*'], 'lang':lang,
                              'newRevId':rev_id, 'newRevDate':new_rev_data['timestamp'], 
                             'newRevUser':diff['touser'], 'newRevComment':diff['toparsedcomment'],
                             'oldRevId':diff['fromrevid'], 'oldRevDate':old_rev_data['timestamp'],
                             'oldRevUser':diff['fromuser'], 'oldRevComment':diff['fromparsedcomment'],
                            'pageTitle':diff['totitle']}
        except mwapi.errors.APIError:
            display_datum = {'editDeleted':True, 'diffHTML':None, 'lang':lang,
                              'newRevId':None, 'newRevDate':None, 'newRevUser':None, 'newRevComment':None,
                             'oldRevId':None, 'oldRevDate':None, 'oldRevUser':None, 'oldRevComment':None,
                            'pageTitle':None}
        display_data.append(display_datum)
    
    return display_data


In [274]:
get_display_data([885656316], 'en')

[{'editDeleted': False,
  'diffHTML': '<tr>\n  <td colspan="2" class="diff-lineno">Line 692:</td>\n  <td colspan="2" class="diff-lineno">Line 692:</td>\n</tr>\n<tr>\n  <td class="diff-marker">&#160;</td>\n  <td class="diff-context"><div>:::::: I pushed an update so these sort of cases won\'t be invited. Thanks again for the sharp eyes. [[User:Maximilianklein|Maximilianklein]] ([[User talk:Maximilianklein|talk]]) 00:48, 19 February 2019 (UTC)</div></td>\n  <td class="diff-marker">&#160;</td>\n  <td class="diff-context"><div>:::::: I pushed an update so these sort of cases won\'t be invited. Thanks again for the sharp eyes. [[User:Maximilianklein|Maximilianklein]] ([[User talk:Maximilianklein|talk]]) 00:48, 19 February 2019 (UTC)</div></td>\n</tr>\n<tr>\n  <td class="diff-marker">&#160;</td>\n  <td class="diff-context"><div>:::::::{{u|Maximilianklein}}, Thanks for all the hard work  [[User:RhinosF1|RhinosF1]]&lt;sup&gt;[[User talk:RhinosF1|(chat)]]&lt;/sup&gt;&lt;small&gt;&lt;sub&gt;[[Us

In [268]:
%debug

> <ipython-input-265-37e460a97833>(10)get_rev_dict()
      8     ret = mwapi_session.get(revids=revids_str, action='query', prop='revisions', rvprop='timestamp')
      9     page_key = list(ret['query']['pages'].keys())[0]
---> 10     rev_data = ['query']['pages'][page_key]['revisions']
     11     if old_rev_id:
     12         return rev_data

ipdb> exit


In [254]:
print(get_diff_html_dict(887315120, mwapi_session = mwapi.Session(host=f'https://en.wikipedia.org', user_agent='civilservant datagathering <max@notconfusing.com>')))

{'toid': 60203063, 'torevid': 887315120, 'tons': 3, 'totitle': 'User talk:Mbrzycki333', 'touser': 'Mbrzycki333', 'touserid': 36069353, 'tocomment': 'adding {{dashboard.wikiedu.org talk course link}}', 'toparsedcomment': 'adding {{dashboard.wikiedu.org talk course link}}', 'next': 887425473, '*': '<tr>\n  <td colspan="2" class="diff-lineno">Line 1:</td>\n  <td colspan="2" class="diff-lineno">Line 1:</td>\n</tr>\n<tr>\n  <td colspan="2" class="diff-empty">&#160;</td>\n  <td class="diff-marker">+</td>\n  <td class="diff-addedline"><div>{{dashboard.wikiedu.org talk course link | course = [[Wikipedia:Wiki_Ed/Saint_Mary\'s_University/Women\'s_and_Gender_Issues_(Spring_2019)]] | slug = Saint_Mary\'s_University/Women\'s_and_Gender_Issues_(Spring_2019) }}</div></td>\n</tr>\n'}


In [244]:
ats_data = mwapi_session.get(revids='885656316|885699923', action='query', prop='revisions', rvprop='user|timestamp')

In [260]:
page_key = list(ats_data['query']['pages'].keys())[0]
ats_data['query']['pages'][page_key]['revisions']

[{'user': 'Bonadea', 'timestamp': '2019-03-01T12:54:27Z'},
 {'user': 'Jtmorgan', 'timestamp': '2019-03-01T18:52:48Z'}]

In [176]:
def num_quality_revisions(user_id, lang):
    """report the number of quality revisions a user has"""
    refresh_user = pd.DataFrame({'user_id':[user_id]})
    all_user_revs = get_quality_edits_of_users(refresh_user, lang)
    return len(all_user_revs)
    
def get_quality_edits_of_users(refresh_users, lang):
    """get all the quality edits of refresh_users stored or live"""
    all_user_revs = get_all_users_revs(refresh_users, lang)
    quality_user_revs = remove_non_quality_revs(all_user_revs)
    return quality_user_revs

def refresh_revisions(refresh_users, lang):
    """assumption we are only refreshing users who are known to need refresh.
    we assume that another process calculates who needs refersh based on their edit count.
    In additon just doing 1 `lang` at a time. So a calling function would have to loop over langs"""
    #do this in a user-oriented way, or a process-oriented way?
    #revisions of users
    all_user_revs = get_quality_edits_of_users(refresh_users, lang)
    #already recevied revisions
    already_revs = get_already_revs(lang)
    #revisions needing getting = revs - already
    revs_to_get = set_subtract(all_user_revs, already_revs)
    #get and store.
    display_data = get_display_data(revs_to_get, lang)

    # return explictly the display data that would need to be sync'd back to server
    return display_data

In [177]:
num_quality_revisions(1420540, 'ar')

started with 6 revs, removed 0


6

In [178]:
num_quality_revisions(697036, 'fa') == 0

started with 1 revs, removed 1


True

In [61]:
num_quality_revisions(855616, 'pl')

started with 40 revs, removed 1


39

In [63]:
get_recent_edits('pl', 851714)

,user_id,rev_timestamp,rev_id
0,851714,2018-01-26 18:12:28,52313950


In [62]:
num_quality_revisions(851714, 'pl')

started with 1 revs, removed 0


1

In [ ]:
num_quality_revisions(2760580, 'de') ==5

In [25]:
if __name__ == "__main__":
    pa_dir = '/home/paprika/Tresors/CivilServant/projects/wikipedia-integration/gratitude-study/datasets/power_analysis/'
    fa_ar_pl_csv = 'gratitude_power-analysis_dataset_sim_date_20180306_v1.csv'
    de_csv = 'de_gratitude_power-analysis_dataset_sim_date_20180306_v1.csv'
    thankees = pd.read_csv(os.path.join(pa_dir, fa_ar_pl_csv), index_col=0)
    thankees_sub = thankees.sample(n=120)
    thankees_de = pd.read_csv(os.path.join(pa_dir, de_csv), index_col=0)
    thankees_de_sub = thankees_de.sample(n=40)

In [118]:
thankees_de_sub['num_quality'] = thankees_de_sub['user_id'].apply(lambda x: num_quality_revisions(x, 'de'))

[180955561 181097334 181241962 181242091 182355581 182355955 182367455
 182367468 182367725 182654022 182654232 182681480 182883712 183096394]
started with 14 revs, removed 0
[167458609 167458641]
started with 2 revs, removed 1
[180399529 180401135 180591876 180592199 180620321]
started with 5 revs, removed 0
[173126360 173481306]
started with 2 revs, removed 0
[177142573 177142709]
started with 2 revs, removed 0
[172074231]
started with 1 revs, removed 1
[180261428 180503423 182506253]
started with 3 revs, removed 0
[183091326 183091343 183091661 183092092 183092555 183092628 183095000
 183095874 183096089 183110365 183211761 183212039 183339666 183340806
 183341784 183378669 183504168 183575437 183686385 183865740 183865769
 183885341 183918242 184027652 184294987 184345784 184430504 184432036
 184432151 184445821 184446221 184446272 184458343 184458659 184459173
 184459205 184464265 184464384 184465358 184465393 184465710 184602415
 184602930 184620138 184620281 184620928 184621828 

In [73]:
thankees_de_sub[['user_id','num_flagged_revisions_pre_treatment']]

,user_id,num_flagged_revisions_pre_treatment
5335936,2498636,9
5537730,2701928,1
5595824,2760580,37
3547236,699386,20
5504857,2668769,0
5646116,2811527,0
4439001,1591684,50
4473700,1626499,1
5667604,2833223,2
5662701,2828211,0


In [43]:
thankees[thankees['num_non_damaging_pre_treatment'] < thankees ['num_goodfaith_pre_treatment']][['user_id','lang','num_goodfaith_pre_treatment','num_non_damaging_pre_treatment']]

,user_id,lang,num_goodfaith_pre_treatment,num_non_damaging_pre_treatment
2142480,697036,fa,1,0
2142455,697011,fa,2,1
2142320,696875,fa,1,0
2142387,696942,fa,1,0
2142461,697017,fa,3,0
2142487,697043,fa,1,0
2142298,696853,fa,1,0
2142315,696870,fa,1,0
2142287,696838,fa,1,0
2142308,696863,fa,1,0
